# Comparing prior volumes with different priors

Notebook to produce a figure comparing nested sampling runs with different priors.

Consider a problem in 1-D with a Gaussian Likelihood.

Consider two priors:

- Truncated Gaussian [-5, 5]
- Uniform [-5, 5]

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from scipy import stats

from thesis_utils.plotting import set_plotting, get_default_figsize, save_figure

set_plotting()

Define the prior distributions

In [ ]:
bounds = 5 * np.array([-1, 1])
prior_tg = stats.truncnorm(*bounds)
prior_u = stats.uniform(bounds[0], np.ptp(bounds))

Define the likelihood in the sample space and in the radius space

In [ ]:
# Likelihood distribution
l_dist = stats.norm()
# Likelihood distribution as a function of r
l_r_dist = stats.chi(df=1)

In [ ]:
def likelihood(r: np.ndarray) -> np.ndarray:
    """Likelihood as a function of radius"""
    return l_r_dist.pdf(r)

Define a function to map between prior volume and radius. This uses the `interval` function from scipy distributions and will only work for symmetric distributions.

In [ ]:
def prior_vol_to_radius(X: np.ndarray, dist: stats.rv_continuous):
    """Convert prior volume to radius for a given distribution"""
    x = np.abs(dist.interval(X))
    assert all(np.abs(x[0] - x[1]) < 1e-6)
    return x[0]

Compute the likelihoods

In [ ]:
X_vec = np.linspace(1.0, 0.0, 100_000, endpoint=True)

r_tg = prior_vol_to_radius(X_vec, prior_tg)
r_u = prior_vol_to_radius(X_vec, prior_u)
samples_tg = np.random.choice([-1, 1], size=len(r_tg)) * r_tg
samples_u = np.random.choice([-1, 1], size=len(r_u)) * r_u
likelihood_tg = likelihood(r_tg)
likelihood_u = likelihood(r_u)

w_i = (X_vec[:-1] - X_vec[1:])

Estimate the evidences

In [ ]:
Z_tg = np.sum(w_i[:-1] * likelihood_tg[1:-1])
Z_u = np.sum(w_i[:-1] * likelihood_u[1:-1])
print(Z_tg, Z_u)

In [ ]:
def reweight(Z_1, P_1, samples, prior_1, prior_2):
    pi_1 =  prior_1.pdf(samples)
    pi_2 = prior_2.pdf(samples)
    Z_re = Z_1 * np.sum(P_1 * (pi_2 /pi_1)[1:-1])
    P_re = P_1 * (Z_1 / Z_re) * (pi_2 / pi_1)[1:-1]
    return Z_re, P_re

Reweighting as described in https://arxiv.org/abs/2205.15570

In [ ]:
P_tg = w_i[:-1] * likelihood_tg[1:-1] / Z_tg
print("Gaussian to uniform")
Z_tg2u, P_tg2u = reweight(Z_tg, P_tg, samples_tg, prior_tg, prior_u)
print(f"True: {Z_u}")
print(f"Estimated: {Z_tg2u}")
print("Uniform to Gaussian")
P_u = w_i[:-1] * likelihood_u[1:-1] / Z_u
Z_u2tg, P_u2tg = reweight(Z_u, P_u, samples_u, prior_u, prior_tg)
print(f"True: {Z_tg}")
print(f"Estimated: {Z_u2tg}")

In [ ]:
hist_kwargs = dict(density=True, histtype="step", bins=100)
# plt.hist(samples_tg[1:-1], weights=P_tg2u, **hist_kwargs)
fig, axs = plt.subplots(1, 3, sharey=True, figsize=(12, 4))

axs[0].hist(samples_tg[1:-1], weights=P_tg, color="C0", **hist_kwargs)
axs[0].hist(samples_u[1:-1], weights=P_u, color="C1", **hist_kwargs)
axs[0].set_title("Initial posteriors")

axs[1].hist(samples_tg[1:-1], weights=P_tg, color="C0", **hist_kwargs)
axs[1].hist(samples_u[1:-1], weights=P_u2tg, color="C1", ls="--", **hist_kwargs)
axs[1].set_title("Uniform to Gaussian")

axs[2].hist(samples_u[1:-1], weights=P_u, color="C1", **hist_kwargs)
axs[2].hist(samples_tg[1:-1], weights=P_tg2u, color="C0", ls="--", **hist_kwargs)
axs[2].set_title("Gaussian to uniform")
plt.show()

Produce the figure

In [ ]:
default_figize = get_default_figsize()
figsize = (default_figize[0], default_figize[1] / 2)

fig, axs = plt.subplots(1, 2, figsize=figsize)

axs[0].plot(X_vec, likelihood_tg, label=f"Truncated Gaussian prior - Z = {Z_tg:.2f}", ls='-')
axs[0].plot(X_vec, likelihood_u, label=f"Uniform prior - Z = {Z_u:.2f}", ls='--')
axs[0].set_xlabel(r"$X$")
axs[0].set_ylabel(r"$\mathcal{L}(X)$")

axs[1].plot(X_vec, r_tg, ls='-')
axs[1].plot(X_vec, r_u, ls='--')
axs[1].set_xlabel(r"$X$")
axs[1].set_ylabel(r"$r$")

fig.legend(
    ncol=2,
    bbox_to_anchor=[0.5, -0.0],
    loc='center',
)
plt.tight_layout()
save_figure(fig, "prior_volume_comparison", "figures")
plt.show()